### Video and Paras
1. Video = "1.mp4" [ "1_x0.5.mp4" ]
roi = (550, 50, 570, 320)
paras = [1.9,100,50,32,11,15]

2. Video = "2.mp4" [ "2_x0.5.mp4" ]
roi = (700, 280, 800, 500)
paras = [1.5,150,45,33,11,15]

3. Video = "" [ "" ]
roi =
paras =

### Project Version: Return Postions

In [ ]:
#Project Version: Frame in, Positions out

#cap = cv2.VideoCapture("baseball pitch5_0.5.mp4")
#fgbg = cv2.createBackgroundSubtractorMOG2()
#ret, frame = cap.read()

import cv2
import numpy as np

def object_detection(frame,roi,paras,sub_on,fgbg=None):
    if sub_on:
        fgmask = fgbg.apply(frame)
        foreground = cv2.bitwise_and(frame, frame, mask=fgmask)
        frame = foreground

    #ROI example:(550, 50, 570, 320)
    x_start, y_start, width, height = roi

    # Crop to the ROI
    roi_frame = frame[y_start:y_start+height, x_start:x_start+width]

    # Convert to grayscale
    gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, 100, 200)

    # Apply Hough Circle Transform
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=paras[0], minDist=paras[1],
                               param1=paras[2], param2=paras[3], minRadius=paras[4], maxRadius=paras[5])

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for i in range(len(circles)):
            # Adjust coordinates to map back to original frame
            (x,y,r) = circles[i]
            circles[i] = (x + x_start, y + y_start, r)

    return(circles)

In [ ]:
#Test Code for Project Version:

cap = cv2.VideoCapture("ProjectVideos/1_x0.5.mp4")
fgbg = cv2.createBackgroundSubtractorMOG2()
roi = (550, 50, 570, 320)  #(x_start, y_start, width, height)
paras=[1.9,100,50,32,11,15]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    pos = object_detection(frame,roi,paras,True,fgbg)
    # print(pos) #(x,y,r)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

### Test Version: Show and Save results

In [ ]:
#Test Version: Video in, show and save the result
import cv2
import numpy as np

# def draw_circles(origin,frame,roi,paras):
#     x_start, y_start, width, height = roi

#     # Crop to the ROI
#     roi_frame = frame[y_start:y_start+height, x_start:x_start+width]

#     # Convert to grayscale
#     gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

#     # Apply Gaussian blur
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#     # Perform Canny edge detection
#     edges = cv2.Canny(blurred, 100, 200)

#     # Apply Hough Circle Transform
#     circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=paras[0], minDist=paras[1],
#                                param1=paras[2], param2=paras[3], minRadius=paras[4], maxRadius=paras[5])

#     if circles is not None:
#         circles = np.round(circles[0, :]).astype("int")
#         for (x, y, r) in circles:
#             cv2.circle(origin, (x + x_start, y + y_start), r, (0, 0, 255), 4)

#     cv2.rectangle(origin, (x_start, y_start), (x_start + width, y_start + height), (255, 0, 0), 3)

#     return origin

# Updated Apr 22
def draw_circles(origin,frame,roi,paras):
    x_start, y_start, width, height = roi

    # Crop to the ROI
    roi_frame = frame[y_start:y_start+height, x_start:x_start+width]

    # Convert to grayscale
    gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (7, 7), 1)

    # Apply thresholding for pixels close to white
    _, thresh = cv2.threshold(blurred, 130, 255, cv2.THRESH_BINARY)

    # For thicker edges
    # dilate = cv2.dilate(thresh, None, iterations=1)

    # Perform Canny edge detection
    edges = cv2.Canny(thresh, 100, 200)

    # Apply Hough Circle Transform
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=paras[0], minDist=paras[1],
                               param1=paras[2], param2=paras[3], minRadius=paras[4], maxRadius=paras[5])

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(origin, (x + x_start, y + y_start), r, (0, 0, 255), 3)

    cv2.rectangle(origin, (x_start, y_start), (x_start + width, y_start + height), (255, 0, 0), 3)

    return origin

def object_detection_save(inpath,outpath,roi,paras,sub_on):
    cap = cv2.VideoCapture(inpath)
    fgbg = cv2.createBackgroundSubtractorMOG2()

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(outpath, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if sub_on:
            fgmask = fgbg.apply(frame)
            foreground = cv2.bitwise_and(frame, frame, mask=fgmask)
            processed_frame = draw_circles(frame,foreground,roi,paras)
        else:
            processed_frame = draw_circles(frame,frame,roi,paras)

        out.write(processed_frame)
    cap.release()
    out.release()

def object_detection_show(inpath,roi,paras,sub_on):
    cap = cv2.VideoCapture(inpath)
    fgbg = cv2.createBackgroundSubtractorMOG2()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if sub_on:
            fgmask = fgbg.apply(frame)
            foreground = cv2.bitwise_and(frame, frame, mask=fgmask)
            processed_frame = draw_circles(frame,foreground,roi,paras)
        else:
            processed_frame = draw_circles(frame,frame,roi,paras)

        cv2.imshow('Detected Circles', processed_frame)
        cv2.waitKey(10) #10 millisecond buffer
        if 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#Test Code for Show-and-Save Version:

# 1
# roi = (350, 50, 150, 150)  #(x_start, y_start, width, height)
# paras=[1.2,10,200,10,3,8]
# path = "ProjectVideos/1.mp4"
# outpath = "1_result.mp4"

# 2
# roi = (1000, 250, 300, 300)  #(x_start, y_start, width, height)
# paras=[1.,30,200,10,13,15]
# path = "ProjectVideos/2_x0.5.mp4"
# outpath = "2_x0.5_result.mp4"

# 3
# roi = (420, 100, 100, 100)  #(x_start, y_start, width, height)
# paras=[1.5,10,200,11,4,10] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
# path = "ProjectVideos/3_curveball.mp4"
# outpath = "3_curveball_result.mp4"

# 4
# roi = (320, 80, 150, 150)  #(x_start, y_start, width, height)
# paras=[1.5,10,200,11,4,8] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
# path = "ProjectVideos/4_4_seam_fastball.mp4"
# outpath = "4_4_seam_fastball_result.mp4"

# 5
roi = (330, 65, 150, 150)  #(x_start, y_start, width, height)
paras=[1.5,10,200,18,2,10] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
path = "ProjectVideos/5_change_up.mp4"
outpath = "5_change_up_result.mp4"

object_detection_show(path,roi,paras,True)
# object_detection_save(path, outpath, roi,paras,True)


2024-04-22 17:34:23.060 Python[29667:8054584] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
